In [10]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import optuna
import shap
import missingno as msno


from datetime import datetime
from catboost import Pool, CatBoostRegressor, EShapCalcType, EFeaturesSelectionAlgorithm
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import mean_squared_error, mean_squared_log_error

ValueError: numpy.dtype size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject

In [11]:
pip install --upgrade pandas pandasai


Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 13.1 MB 200 kB/s eta 0:00:01
     |████████████████████████████████| 193 kB 484 kB/s eta 0:00:01
     |████████████████████████████████| 1.7 MB 262 kB/s eta 0:00:01
     |████████████████████████████████| 435 kB 442 kB/s eta 0:00:01
     |████████████████████████████████| 20.1 MB 58 kB/s  eta 0:00:01
     |████████████████████████████████| 797.1 MB 1.5 kB/s eta 0:00:012    |██████▉                         | 171.1 MB 1.1 MB/s eta 0:09:12     |█████████████▉                  | 345.3 MB 192 kB/s eta 0:39:08     |████████████████████████▎       | 604.8 MB 155 kB/s eta 0:20:37     |██████████████████████████▋     | 661.5 MB 112 kB/s eta 0:20:08     |█████████████████████████████▍  | 731.7 MB 218 kB/s eta 0:05:00
     |████████████████████████████████| 12.1 MB 722 kB/s eta 0:00:01
     |████████████████████████████████| 23.7 MB 61 kB/s eta 0:00:012    |██████▊                